In [16]:
from galactic import GalacticDataset
import pandas as pd

In [17]:
# downloaded from https://huggingface.co/datasets/openchat/openchat_sharegpt4_dataset/tree/main
ds = pd.read_json("/Users/ben/Downloads/sharegpt_gpt4.json", orient="records") # extra step because Galactic uses JSONL but not JSON, JSON is cursed
ds = GalacticDataset.from_pandas(ds)
ds['items'][0]

Map:   0%|          | 0/6124 [00:00<?, ? examples/s]

[{'from': 'human',
  'value': "I want to send a custom PHP file url to a person and once he has visited the URL, I don't want the url to subsequently open on any other device except the one it was opened first on. How do I implement this?"},
 {'from': 'gpt',
  'value': 'To achieve this, you can implement a one-time URL access system with PHP, which stores the device\'s fingerprint when the URL is first accessed and then checks for a match in subsequent accesses. Here\'s an outline of the process:\n\n1. Generate a unique ID for the URL and save it in a database along with an initial "unused" status.\n2. When the user visits the URL, check the status of the unique ID in the database.\n3. If the URL is unused, save the device\'s fingerprint in the database and update the URL status to "used."\n4. If the URL has been used, check if the device fingerprint matches the one saved in the database. If it matches, grant access, otherwise deny access.\n\nHere\'s some sample code to help you get st

As you can see, we've already got a column that has a nice list of messages, but it's not formatted in a standard way (the "role" field is called "from", the "content" field is "value", and the roles are "human" and "gpt"). Let's fix that.

In [18]:
# tell the function what fields are named in the input data
input_message_config = {
    "role_field": "from",
    "content_field": "value",
    "user_role": "human",
    "assistant_role": "gpt",
    "system_role": "system",
}

ds.conversation_from_dicts(
    input_column="items",
    input_message_config=input_message_config,
    output_column="conversation",
    # output_message_config -- if omitted, will automatically use the OpenAI "role", "content", "user", "assistant" "system" format
    # conversation_field -- only needed if the list of messages is buried in a sub-field of the input_column
    metadata_fields=[],
)
ds["conversation"][0]

Map:   0%|          | 0/6124 [00:00<?, ? examples/s]

[{'content': "I want to send a custom PHP file url to a person and once he has visited the URL, I don't want the url to subsequently open on any other device except the one it was opened first on. How do I implement this?",
  'metadata': {},
  'role': 'user'},
 {'content': 'To achieve this, you can implement a one-time URL access system with PHP, which stores the device\'s fingerprint when the URL is first accessed and then checks for a match in subsequent accesses. Here\'s an outline of the process:\n\n1. Generate a unique ID for the URL and save it in a database along with an initial "unused" status.\n2. When the user visits the URL, check the status of the unique ID in the database.\n3. If the URL is unused, save the device\'s fingerprint in the database and update the URL status to "used."\n4. If the URL has been used, check if the device fingerprint matches the one saved in the database. If it matches, grant access, otherwise deny access.\n\nHere\'s some sample code to help you ge

Now, we want to generate a dataset for RLHF. We'll use an RLAIF approach: get a critique from a LLM, then use that critique to re-write the last assistant response to be better. We'll use several Galactic utilities along the way. First, we'll make sure all the conversations end with an assistant message. Then, we'll split off that last message, and turn both the previous conversation and the last message into strings that can be used to prompt an LLM.

In [19]:
# now that we've fixed the format, the input_message_config is the OpenAI format
input_message_config = {
    "role_field": "role",
    "content_field": "content",
    "user_role": "user",
    "assistant_role": "assistant",
    "system_role": "system",
}
ds = ds.get_last_speaker(
    "conversation",
    input_message_config=input_message_config,   
)
from collections import Counter
Counter(ds["__last_speaker"])



Map:   0%|          | 0/6124 [00:00<?, ? examples/s]

Counter({'assistant': 6123, 'user': 1})

We've only got 1 conversation where the last speaker isn't the assistant, but with messier datasets you might expect more cases like this. Luckily we have a utility to make sure the last speaker of every conversation is the assistant. Then, we'll strip off that last assistant message and put it in a separate column.

In [20]:
ds.standardize_last_turn(
    "conversation",
    input_message_config=input_message_config,
    last_speaker_role="assistant"
)
ds = ds.get_last_speaker(
    "conversation",
    input_message_config=input_message_config,   
)
Counter(ds["__last_speaker"])

Map:   0%|          | 0/6124 [00:00<?, ? examples/s]

Map:   0%|          | 0/6124 [00:00<?, ? examples/s]

Counter({'assistant': 6124})

In [21]:
ds.take_last_message(
    input_column="conversation",
    input_message_config=input_message_config,
    output_column="last_message",
    output_type="string",
    remove_from_input=True,
);
ds["last_message"][0]

Map:   0%|          | 0/6124 [00:00<?, ? examples/s]

'To use a more robust fingerprinting library like FingerprintJS, you\'ll need to modify the previous implementation by adding client-side JavaScript to generate the device fingerprint and then send it to the server. Here\'s how you can do it:\n\n1. Include the FingerprintJS library in your project. You can either download it from <https://github.com/fingerprintjs/fingerprintjs> or use the CDN link:\n```html\n<script src="https://cdnjs.cloudflare.com/ajax/libs/fingerprintjs/3.3.0/fp.min.js"></script>\n```\n1. Modify the `one-time-url.php` file to include the FingerprintJS library, use JavaScript to get the fingerprint, and send it to the server using AJAX:\n```php\n<?php\n// one-time-url.php\nif (!isset($_GET[\'uid\'])) {\n  die(\'Invalid URL\');\n}\n\n$unique_id = $_GET[\'uid\'];\n?>\n<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta charset="UTF-8">\n  <meta name="viewport" content="width=device-width, initial-scale=1.0">\n  <title>One-Time URL</title>\n  <script src="https://cdnjs.c

Looks good! Now, we'll add the "Constitutional AI" style pipeline. We'll first ask for a critique of the last reply. First, however, we should make the "previous conversation" a string instead of a JSON list, and also filter ones that are too long so we don't exceed the LLM's context length (and so it doesn't take too long to run).

In [22]:
ds = ds.convert_conversation_to_string(
    input_column="conversation",
    input_message_config=input_message_config,
    output_column="conversation_string",
    user_delimiter="\n\nUser: ",
    assistant_delimiter="\n\nAssistant: ",
    system_delimiter="\n\nSystem: ",
).count_tokens(
    fields=["conversation_string"],
    tokenizer="gpt2", # close enough
).filter(
    lambda x: x["__token_count__conversation_string"] < 2500
)
len(ds)

Map:   0%|          | 0/6124 [00:00<?, ? examples/s]

Map:   0%|          | 0/6124 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1064 > 1024). Running this sequence through the model will result in indexing errors
INFO: Counted tokens in fields: ['conversation_string'], added metadata to __token_count__


Filter:   0%|          | 0/6124 [00:00<?, ? examples/s]

3724

In [23]:
import nest_asyncio # have to do this to do API requests from within a notebook
nest_asyncio.apply()

prompt = (
    "Below is a JSON representation of a conversation between an AI assistant and a human, including the AI's final reply. "
    "The ideal AI reply is helpful, relevant to the user's request, and as short & to-the-point as possible, while "
    "avoiding providing any dangerous or harmful information. Please describe where the AI's reply falls short according to "
    "these criteria, in short bullets. If you think the AI's reply meets the criteria, don't list reasons, just write 'Perfect' or 'Looks good'. "
    "Do not provide any positive feedback--only criticism. If you realize you can't think of serious criticism, just say it looks good."
    "\n\n---\n\nPrevious conversation:\n{{conversation_string}}\n\n---\n\nAI's reply:\n{{last_message}}\n\n---\n\nCritique:"
)
ds.set_openai_key(open("/Users/ben/Desktop/oai_api_key.txt").read().strip())
ds.set_rate_limits(
    max_tokens_per_minute=400_000,
    max_requests_per_minute=4_000
)
ds.ai_column(
    new_column="critique",
    prompt=prompt,
    depends_on=["conversation_string", "last_message"],
)

Map:   0%|          | 0/3724 [00:00<?, ? examples/s]

INFO: Example prompt: Below is a JSON representation of a conversation between an AI assistant and a human, including the AI's final reply. The ideal AI reply is helpful, relevant to the user's request, and as short & to-the-point as possible, while avoiding providing any dangerous or harmful information. Please describe where the AI's reply falls short according to these criteria, in short bullets. If you think the AI's reply meets the criteria, don't list reasons, just write 'Perfect' or 'Looks good'. Do not provide any positive feedback--only criticism. If you realize you can't think of serious criticism, just say it looks good.

---

Previous conversation:
User: I want to send a custom PHP file url to a person and once he has visited the URL, I don't want the url to subsequently open on any other device except the one it was opened first on. How do I implement this?

Assistant: To achieve this, you can implement a one-time URL access system with PHP, which stores the device's finge

  0%|          | 0/3724 [00:00<?, ?it/s]

ERROR: Exception: 
ERROR: Exception: 
ERROR: Exception: 
INFO: Parallel processing complete.


Flattening the indices:   0%|          | 0/3724 [00:00<?, ? examples/s]

INFO: Added new column critique using prompt Below is a JSON representation of a conversation between an AI assistant and a human, including the AI's final reply. The ideal AI reply is helpful, relevant to the user's request, and as short & to-the-point as possible, while avoiding providing any dangerous or harmful information. Please describe where the AI's reply falls short according to these criteria, in short bullets. If you think the AI's reply meets the criteria, don't list reasons, just write 'Perfect' or 'Looks good'. Do not provide any positive feedback--only criticism. If you realize you can't think of serious criticism, just say it looks good.

---

Previous conversation:
{{conversation_string}}

---

AI's reply:
{{last_message}}

---

Critique:


                                               items         model  \
0  [{'from': 'human', 'value': 'I want to send a ...  Model: GPT-4   
1  [{'from': 'human', 'value': 'Explain the plot ...  Model: GPT-4   
2  [{'from': 'human', 'value': 'I have a folder c...  Model: GPT-4   
3  [{'from': 'human', 'value': 'I envision a plug...  Model: GPT-4   
4  [{'from': 'human', 'value': 'I wish to prepare...  Model: GPT-4   
5  [{'from': 'human', 'value': 'Dado este XML:


...  Model: GPT-4   
6  [{'from': 'human', 'value': 'I'm using react-n...  Model: GPT-4   
7  [{'from': 'human', 'value': 'for the sample si...  Model: GPT-4   
8  [{'from': 'human', 'value': 'write a markdown ...  Model: GPT-4   
9  [{'from': 'human', 'value': '如何开发一个屏蔽百度广告的chro...  Model: GPT-4   

                                           avatarUrl       id  views  \
0  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...  oSX74tm     15   
1  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...  mwN7As7      1   
2  data:image

We'll filter out the ones where the critique mentions "perfect", as these are unlikely to spur improvement.

In [24]:
ds = ds.filter(
    lambda x: "perfect" not in x["critique"].lower() and "looks good" not in x["critique"].lower()
)
len(ds)

Filter:   0%|          | 0/3724 [00:00<?, ? examples/s]

2157

Great, now based on the critique, we'll use another API call to generate an improved response in a new column. To keep it fast, we'll just do this for 1000 of the critiques.

In [25]:
prompt = (
    "Below is a conversation between an AI assistant and a human. The AI's last reply is accompanied by a critique. "
    "Your job is to revise the AI's reply to address the critique. Please only respond with the revised reply, and nothing else. "
    "The reply should be concise, and improve the response according to the feedback in the critique.\n\n---\n\nPrevious conversation:\n"
    "{{conversation_string}}\n\n---\n\nAI's reply:\n{{last_message}}\n\n---\n\nCritique:\n{{critique}}\n\n---\n\nRevised reply:"
)
with_revisions = ds.select(range(1000)).ai_column(
    new_column="revised_reply",
    prompt=prompt,
    depends_on=["conversation_string", "last_message", "critique"],
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

INFO: Example prompt: Below is a conversation between an AI assistant and a human. The AI's last reply is accompanied by a critique. Your job is to revise the AI's reply to address the critique. Please only respond with the revised reply, and nothing else. The reply should be concise, and improve the response according to the feedback in the critique.

---

Previous conversation:
User: I want to send a custom PHP file url to a person and once he has visited the URL, I don't want the url to subsequently open on any other device except the one it was opened first on. How do I implement this?

Assistant: To achieve this, you can implement a one-time URL access system with PHP, which stores the device's fingerprint when the URL is first accessed and then checks for a match in subsequent accesses. Here's an outline of the process:

1. Generate a unique ID for the URL and save it in a database along with an initial "unused" status.
2. When the user visits the URL, check the status of the uni

  0%|          | 0/1000 [00:00<?, ?it/s]

ERROR: Exception: 
ERROR: Exception: 
ERROR: Exception: 
ERROR: Exception: 
INFO: Parallel processing complete.


Flattening the indices:   0%|          | 0/1000 [00:00<?, ? examples/s]

INFO: Added new column revised_reply using prompt Below is a conversation between an AI assistant and a human. The AI's last reply is accompanied by a critique. Your job is to revise the AI's reply to address the critique. Please only respond with the revised reply, and nothing else. The reply should be concise, and improve the response according to the feedback in the critique.

---

Previous conversation:
{{conversation_string}}

---

AI's reply:
{{last_message}}

---

Critique:
{{critique}}

---

Revised reply:


Now, we have a collection of conversations, with a "dispreferred" (initial) reply, and a "preferred" (revised) reply. We can use this to generate a dataset for RLHF-style training.

In [26]:
for i in range(3):
    print("## Example " + str(i))
    print("[INITIAL LAST MESSAGE] " + with_revisions["last_message"][i])
    print("[CRITIQUE] " + with_revisions["critique"][i])
    print("[REVISION] " + with_revisions["revised_reply"][i])
    print()

## Example 0
[INITIAL LAST MESSAGE] To use a more robust fingerprinting library like FingerprintJS, you'll need to modify the previous implementation by adding client-side JavaScript to generate the device fingerprint and then send it to the server. Here's how you can do it:

1. Include the FingerprintJS library in your project. You can either download it from <https://github.com/fingerprintjs/fingerprintjs> or use the CDN link:
```html
<script src="https://cdnjs.cloudflare.com/ajax/libs/fingerprintjs/3.3.0/fp.min.js"></script>
```
1. Modify the `one-time-url.php` file to include the FingerprintJS library, use JavaScript to get the fingerprint, and send it to the server using AJAX:
```php
<?php
// one-time-url.php
if (!isset($_GET['uid'])) {
  die('Invalid URL');
}

$unique_id = $_GET['uid'];
?>
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>One-Time URL</title>
  <script src="https://cd

In [27]:
# all we have to do is add an "\n\nAssistant:" to the conversation string, and space to the start of the replies, 
# so that the replies are a natural continuation for ML training.
rlhf_dataset = with_revisions.select_columns(["conversation_string", "last_message", "revised_reply"]).map(
    lambda x: {
        "conversation_string": x["conversation_string"] + "\n\nAssistant:",
        "last_message": " " + x["last_message"],
        "revised_reply": " " + x["revised_reply"]
    }
)
rlhf_dataset[0]


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'conversation_string': 'User: I want to send a custom PHP file url to a person and once he has visited the URL, I don\'t want the url to subsequently open on any other device except the one it was opened first on. How do I implement this?\n\nAssistant: To achieve this, you can implement a one-time URL access system with PHP, which stores the device\'s fingerprint when the URL is first accessed and then checks for a match in subsequent accesses. Here\'s an outline of the process:\n\n1. Generate a unique ID for the URL and save it in a database along with an initial "unused" status.\n2. When the user visits the URL, check the status of the unique ID in the database.\n3. If the URL is unused, save the device\'s fingerprint in the database and update the URL status to "used."\n4. If the URL has been used, check if the device fingerprint matches the one saved in the database. If it matches, grant access, otherwise deny access.\n\nHere\'s some sample code to help you get started:\n\n1. Crea

In [29]:
rlhf_dataset.save("/Users/ben/Desktop/rlhf_dataset.jsonl", overwrite=True)